# Spark SQL Exercise

Let's work with some airline data and play around/do some EDA.

In [1]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 199 kB 50.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=292a4a235e428e063a053cbe0aeeee700e4cd906aff675cf96309f6ddb70bfae
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

airports_path = '/content/drive/MyDrive/Colab Notebooks/Big_data/day_1/data/airport_codes.txt'
flight_delay_path = '/content/drive/MyDrive/Colab Notebooks/Big_data/day_1/data/US_Flights_2013.csv'

**Let**'s load the data from the correct path and also make it into a view that SparkSQL and query.

In [4]:
# Obtain Airports dataset
airports = spark.read.csv(airports_path, header='true', inferSchema='true',
                          sep='\t')
airports.createOrReplaceTempView("airports")

# Obtain Departure Delays dataset
flightPerf = spark.read.csv(flight_delay_path, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

In [5]:
flightPerf.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- OriginAirportName: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DestAirportName: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DestState: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- DepDel15: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- ArrDel15: string (nullable = true)
 |-- Cancelled: string (nullable = true)



In [6]:
airports.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



In [7]:
spark.sql('SELECT CITY, IATA FROM airports WHERE state="IL" ').show()

+-----------+----+
|       CITY|IATA|
+-----------+----+
|Bloomington| BMI|
|  Champaign| CMI|
|    Chicago| MDW|
|    Chicago| ORD|
|    Chicago| CHI|
|    Decatur| DEC|
|     Marion| MWA|
|     Moline| MLI|
|     Peoria| PIA|
|     Quincy| UIN|
|   Rockford| RFD|
|Springfield| SPI|
+-----------+----+



In [8]:
spark.sql('SELECT * FROM FlightPerformance').show(5)

+----+-----+----------+---------+-------+---------------+--------------------+----------+-----------+-------------+--------------------+----------+---------+----------+--------+--------+----------+--------+--------+---------+
|Year|Month|DayofMonth|DayOfWeek|Carrier|OriginAirportID|   OriginAirportName|OriginCity|OriginState|DestAirportID|     DestAirportName|  DestCity|DestState|CRSDepTime|DepDelay|DepDel15|CRSArrTime|ArrDelay|ArrDel15|Cancelled|
+----+-----+----------+---------+-------+---------------+--------------------+----------+-----------+-------------+--------------------+----------+---------+----------+--------+--------+----------+--------+--------+---------+
|2013|    9|        16|        1|     DL|          15304| Tampa International|     Tampa|         FL|        12478|John F. Kennedy I...|  New York|       NY|      1539|       4|       0|      1824|      13|       0|        0|
|2013|    9|        23|        1|     WN|          14122|Pittsburgh Intern...|Pittsburgh|       

In [ ]:
query = '''
SELECT DestState, avg(DepDelay) as Delay_minutes FROM FlightPerformance  
   WHERE OriginState="WA" 
   GROUP BY DestState
   ORDER BY DestState ASC
'''

spark.sql(query).show()

+---------+------------------+
|DestState|     Delay_minutes|
+---------+------------------+
|       AK|2.9130434782608696|
|       AZ| 4.231231231231231|
|       CA| 6.105812220566319|
|       CO| 6.888268156424581|
|       DC|11.396396396396396|
|       FL| 9.183098591549296|
|       GA|17.418079096045197|
|       HI| 8.659722222222221|
|       IL| 13.55609756097561|
|       MA|10.647727272727273|
|       MD|14.235294117647058|
|       MI|10.119047619047619|
|       MN|10.323353293413174|
|       MO|2.5348837209302326|
|       NC|              8.25|
|       NJ| 8.725806451612904|
|       NM|             4.625|
|       NV|3.4018691588785046|
|       NY|             5.875|
|       OH|5.9411764705882355|
+---------+------------------+
only showing top 20 rows



## Exercise 1: 

What is the average delay for flights coming _into_ WA?

In [22]:
query = '''
SELECT DestState, avg(ArrDelay) as Delay_minutes FROM FlightPerformance  
   WHERE OriginState="WA" 
   GROUP BY DestState
   ORDER BY DestState ASC
'''

spark.sql(query).show()

+---------+-------------------+
|DestState|      Delay_minutes|
+---------+-------------------+
|       AK|-1.7690217391304348|
|       AZ|  1.954954954954955|
|       CA|   4.78936910084451|
|       CO|  5.148044692737431|
|       DC|   6.63963963963964|
|       FL|-1.7464788732394365|
|       GA| 11.937853107344633|
|       HI|  4.451388888888889|
|       IL| 10.707317073170731|
|       MA|              8.125|
|       MD| 3.5294117647058822|
|       MI|  7.488095238095238|
|       MN|  6.592814371257485|
|       MO| 2.9069767441860463|
|       NC|          10.515625|
|       NJ|  2.314516129032258|
|       NM|            0.96875|
|       NV|  2.545171339563863|
|       NY|  3.908333333333333|
|       OH|-1.2352941176470589|
+---------+-------------------+
only showing top 20 rows



# Exercise 2:
What is the average delay for flights leaving from any airport in Illinois, broken up by destination?

In [35]:
query = '''
SELECT DestCity, avg(DepDelay) as Delay_minutes FROM FlightPerformance  
   WHERE OriginCity="Illinois" 
   GROUP BY DestCity
   ORDER BY DestCity ASC
'''

spark.sql(query).show()

+--------+-------------+
|DestCity|Delay_minutes|
+--------+-------------+
+--------+-------------+

